In [1]:
param = str(input("Introduce año entre los siguientes: 2018, 1996 y 1975: "))
param2 = str(input("Introduce Term o Purpose "))
#por si acaso no lo pone literal
ok = False
while(not ok):
    if (param2.lower() == 'term'):
        param2 = 'Term'
        ok = True
    elif (param2.lower() == 'purpose'):   
        param2 = 'Purpose'
        ok = True
    else:
        print("Dato incorrecto")
        param2 = str(input("Introduce Term o Purpose "))

Introduce año entre los siguientes: 2018, 1996 y 1975: 1975
Introduce Term o Purpose Purpose


In [2]:
import csv
%precision 2
#leemos todos los datos del fichero
with open('datos' + param + '.csv') as csvfile:
    cites = list(csv.DictReader(csvfile, delimiter=';'))

In [3]:
cites[1] #enseñamos como se mostraria cada linea, se ve que hay que separar mas los datos

OrderedDict([('Year', '1975'),
             ('App.', 'I'),
             ('Taxon', 'Melanosuchus niger'),
             ('Class', 'Reptilia'),
             ('Order', 'Crocodylia'),
             ('Family', 'Alligatoridae'),
             ('Genus', 'Melanosuchus'),
             ('Importer', 'CH'),
             ('Exporter', 'FR'),
             ('Origin', ''),
             ('Importer reported quantity', '50'),
             ('Exporter reported quantity', ''),
             ('Term', 'skins'),
             ('Unit', ''),
             ('Purpose', ''),
             ('Source', '')])

In [4]:
#Los conjuntos de nodos posibles son: taxon, familia y order. Y el otro purpose y terms.
#Vamos a usar purpose y terms con un conjunto de nodos animales, y veremos cual es el mejor
family_nodes = set()
taxon_nodes = set()
order_nodes = set()
purpose_nodes = set()
term_nodes = set()
for elem in cites:
    family_nodes.add(elem['Family'])
    order_nodes.add(elem['Order'])
    taxon_nodes.add(elem['Taxon'])
    purpose_nodes.add(elem['Purpose'])
    term_nodes.add(elem['Term'])
    
print(len(family_nodes))
print(len(taxon_nodes))
print(len(order_nodes))
print(len(purpose_nodes))
print(len(term_nodes))

#como podemos ver, hay mas taxon que familia y orden, por lo que decidimos hacer que los nodos 'animales' sean taxon.
#hacemos dos grafos, uno con taxon-term y otro con taxon-purpose

36
85
20
7
13


In [5]:
#grafo bipartito de taxon y terms
%matplotlib notebook
import networkx as nx
from networkx.algorithms import bipartite
from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt

BG = nx.Graph()

for elem in cites:
    if(elem[param2] != '' and elem['Taxon'] != ''): #filtramos ya que hay vacios
        BG.add_node(elem['Taxon'], bipartite=0)
        BG.add_node(elem[param2], bipartite=1)
        BG.add_edge(elem[param2], elem['Taxon'])


In [6]:
print(BG.edges)

[('Elephas maximus', 'Q'), ('Equus przewalskii', 'B'), ('Equus przewalskii', 'Z'), ('B', 'Ciconia nigra'), ('Z', 'Panthera pardus'), ('Z', 'Panthera uncia'), ('Z', 'Pongo pygmaeus'), ('Z', 'Anodorhynchus leari'), ('Z', 'Leucopsar rothschildi'), ('Z', 'Tapirus indicus'), ('Z', 'Ailurus fulgens'), ('Z', 'Falco biarmicus'), ('Z', 'Panthera tigris'), ('Z', 'Pan troglodytes'), ('Z', 'Diceros bicornis'), ('Grus americana', 'S'), ('Lophura swinhoii', 'P'), ('P', 'Syrmaticus humiae'), ('P', 'Varanus bengalensis'), ('P', 'Caiman crocodilus'), ('P', 'Branta ruficollis'), ('P', 'Dendrocygna arborea'), ('P', 'Probosciger aterrimus'), ('P', 'Polihierax semitorquatus'), ('P', 'Leopardus pardalis'), ('P', 'Leopardus wiedii'), ('P', 'Heloderma horridum'), ('P', 'Polyplectron bicalcaratum'), ('P', 'Eunymphicus cornutus'), ('P', 'Prosopeia personata'), ('P', 'Python curtus'), ('P', 'Varanus salvator'), ('Boa constrictor', 'T'), ('T', 'Eunectes notaeus'), ('T', 'Tanygnathus lucionensis'), ('T', 'Python b

In [7]:
nx.is_bipartite(BG)

True

In [8]:
#hacemos la lista de nodos para que gephi pueda leer el archivo correctamente
#la columna bipartire es para poder visualizarlo mejor en gephi, con colores dependiendo de que conjunto pertenece
f=open("node_list_" + param + "_taxon-" + param2 + ".csv","w", encoding = "utf-8")
f.write("ID\tLabel\tBipartito\n")
cont=1
mapa = {}
for n, d in BG.nodes(data=True):
    f.write(str(cont) + "\t" + n + "\t" + str(d['bipartite']) + "\n")
    mapa[n] = cont
    cont = cont + 1
f.close() 

In [9]:
#hacemos la lista de aristas para que gephi pueda leer el archivo correctamente
f=open("edge_list_" + param + "_taxon-" + param2 + ".csv","w", encoding = "utf-8")
f.write("Source\tTarget\n")
for e in BG.edges:
    f.write(str(mapa[e[0]]) + "\t" + str(mapa[e[1]]) + "\n")
f.close()